### Pre-processing and significance testing of Organellar IPs in uninfected state

#### This is the third step of the Organellar IPs analysis

Some Org-IP samples may have very similar profiles, which may lead to underestimation of significance if outlier proteins overlap. 

To tackle this problem, enrichment values are pre-calculated using every other sample in the batch, and these enrichment values are correlated across samples.

The correlation values are then used to filter out highly correlated org-IP samples from being used in the significance test together.

This notebook conducts a sweep of the correlation cutoff values, using the ARI as a evaluating metric 

In [1]:
import os, sys
import sys
from pathlib import Path
import pandas as pd

script_path = Path.cwd().parent.parent.parent.parent / "script"
data_path = Path.cwd().parent.parent.parent.parent / "data"
sys.path.append(str(script_path))
from pyseus import primary_analysis as pa
from pyseus import spatial_tools as st

### Load the imputed IP table
The correct datestamp is required to find the input files

In [2]:
%store -r fig5_timestamp FIG5_USE_FROZEN
if FIG5_USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from 3.aligned_umap")
timestamp = fig5_timestamp
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

Timestamp: 2023-12-04


In [3]:
#### Load bait imputed tables (IPs)
IP_path = preprocessing_out / f"{outprefix}_imputed_table.csv"

try:
    bait_imputed_table = pd.read_csv(IP_path, header=[0, 1], index_col=0)
except FileNotFoundError:
    print(f"File {IP_path} not found.\nPlease run 1.QC_filter_and_impute.ipynb first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {IP_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Correlation filter

In [4]:
batches = [["14", "12", "17"], ["09", "10", "11"]]

# imputed, preprocessed table from the earlier steps
grouped = bait_imputed_table.copy()

# initiate lists to add dataframes for concatenation at the end
volcano_tables = []
enrichment_tables = []
volcano_tables_againstWT = []
enrichment_tables_againstWT = []

# create a folder to save the correlation tables
correlation_table_dir = outdir / "correlation_tables"
os.makedirs(correlation_table_dir, exist_ok=True)

# iterate through batches
for i, batch in enumerate(batches):
    print("batch: " + str(i))
    new_cols = []
    # select all the samples from each batch
    for col in list(grouped):
        if col[0] == "metadata":
            new_cols.append(col)
        # identifying each batch
        elif (col[1].split("-")[0] in batch):  # splitting the dash and seeing if that experiment is in the batch youre running
            new_cols.append(col)
    selected = grouped[new_cols].copy()

    # print(list(selected))
    # initial calculation of complement enrichments to identify highly correlated samples
    analysis = pa.AnalysisTables(grouped_table=selected, auto_group=False)
    analysis.generate_export_bait_matrix()

    # analysis.select_wildtype_controls(wt_re='-WT') # select only WT as controls, this is onfirmed by Manu to be the better than using all samples as the intial controls
    analysis.select_matching_WT_control(wt_re="-WT")  # select matching WT as controls.

    # print(analysis.exclusion_matrix) # uncomment to check the exclusion matrix
    analysis.exclusion_matrix.to_csv(correlation_table_dir / f"{timestamp}_negative_control_batch_{i}.csv", index=False)

    analysis.simple_pval_enrichment(std_enrich=False)
    analysis.convert_to_enrichment_table()
    complements = analysis.enrichment_table.copy()
    control_mat = analysis.exclusion_matrix.copy()

    enrichment_tables_againstWT.append(analysis.enrichment_table.copy())  # export the enrichments against WT
    volcano_tables_againstWT.append(analysis.simple_pval_table.copy())  # export the pvals against WT

    # Using the previous calculations, find highly correlated sample (filter is at >0.35)
    spatial = st.SpatialTables(
        preprocessed_table=selected,
        enrichment_table=complements,
        control_mat=control_mat,
    )
    spatial.enrichment_corr_control_mat(corr=0.35)

    # save the correlation table for each batch
    spatial.corr_mat.to_csv(correlation_table_dir / f"{timestamp}_corr_table_batch_{i}.csv", index=False)
    spatial.sample_corrs.to_csv(correlation_table_dir / f"{timestamp}_corr_val_table_batch_{i}.csv", index=True)

    # calculate the final pval and enrichment table for the batch
    spatial.new_corr_ARI(just_enrichment=True, labels=None, reference=None, std_enrich=False)

    # appending respective dataframes to the concatenation list
    volcano_tables.append(spatial.corr_pval_table)
    enrichment_tables.append(spatial.corr_enrichment_table)

batch: 0
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 1
P-val calculations..
Finished!
P-val calculations..
Finished!


The final step is to concatenate results from the batches. 

The enrichment table can be used for multiple purposes, including UMAP generation.

In [5]:
# concatenating the final enrichment table
enrichment_metadata = enrichment_tables[0].loc[:, ["metadata"]].copy()
enrichment_samples = [x.drop(["metadata"], axis=1, level=0) for x in enrichment_tables]

final_enrichments = pd.concat([enrichment_metadata] + enrichment_samples, axis=1)

# concatenating the final volcano table
volcano_metadata = volcano_tables[0].loc[:, ["metadata"]].copy()
volcano_samples = [x.drop(["metadata"], axis=1, level=0) for x in volcano_tables]

final_volcano = pd.concat([volcano_metadata] + volcano_samples, axis=1)

In [6]:
# saving the final tables to files
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
os.makedirs(enrich_out_dir, exist_ok=True)

volcano_csv_path = enrich_out_dir / f"{timestamp}_volcano_table.csv"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"

final_volcano.to_csv(volcano_csv_path)
final_enrichments.to_csv(enrichment_csv_path)

In [7]:
final_enrichments

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8587                                             X5D7P8   
8588                                             X5D8X9   
8589  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8590                                             X5DQZ7   
8591                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8587                                             X5D7P8            STK39   
8588                                             X5D8X9          CNTNAP2   
8589  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8590                                             X5DQZ7             GPX1   
8591                                             X6RHX1            SCYL3   

        sample                                                                \
     14-GOLGA2  14-RAB14 17-ATP1B3 14-RAB11A 17-MAP1LC3B  12-LAMP1     12-WT   
0     2.153417  1.892810 -0.572400  1.456910    2.979922 -1.784115 -1.925398   
1     0.023500 -0.330450 -1.834500 -1.285350    0.131400 -0.118800  1.427950   
2     4.584300  4.124300 -3.557200  4.259000    4.301950 -1.206450 -5.139850   
3     0.454289  0.499897  0.578235  0.382422   -0.555647  0.213112 -0.135801   
4     2.469750  2.903350 -1.687400  2.420950    3.002300  0.968700 -6.612522   
...        ...       ...       ...       ...         ...       ...       ...   
8587 -0.363124 -0.845920  0.500627  0.603359   -0.009725  0.045149 -0.478913   
8588  2.571864  3.155311  3.373042  2.671111   -1.736598 -2.082729 -1.978545   
8589  0.766317  0.519500 -0.460152  0.239464   -1.826895  0.405054  0.481500   
8590  2.632244  3.655411 -0.809530  3.298411    1.251901 -0.055754 -0.293289   
8591 -0.951167  0.566475 -0.388080  0.224285    0.850954  0.413169  0.982133   

      ...                                                              \
      ... 10-TOMM20     10-WT  10-EXOC2  10-AP2B1 11-GPR107 11-CEP350   
0     ... -0.090828  2.577573 -0.417673  0.569077 -1.941187 -0.501610   
1     ... -2.969400  1.972000  1.017250 -0.336600 -2.043600  0.895300   
2     ... -2.685600  0.006200 -4.497900  0.620900  5.885700  0.397300   
3     ...  0.658883 -0.197179  1.054934  0.517645 -0.749186 -0.364061   
4     ... -2.382900 -0.608600  0.033350  1.823800  5.827450 -0.438800   
...   ...       ...       ...       ...       ...       ...       ...   
8587  ...  1.673807 -0.074423 -0.156704  0.122412  0.041333  0.087444   
8588  ... -1.020045  1.013505 -0.939473 -0.985532 -0.133089  2.203500   
8589  ... -0.156949 -0.751550  0.398087 -0.328330 -0.351563 -0.076900   
8590  ...  3.274953  0.134784 -0.463179 -0.878981  0.200192  0.969230   
8591  ...  0.721173  0.024608 -0.804902 -0.320245 -0.447988  0.003678   

                                                
      09-PSMB7   09-EDC4 09-HSP90AA1   10-RTN4  
0     0.407900  2.150198   -1.514841  0.252975  
1    -0.327100  1.614550   -2.443700 -1.899050  
2    -5.327942 -0.149000   -5.032831  1.852200  
3     1.318186  0.035807   -0.054300 -0.000587  
4    